In [1]:
import os
import cloudscraper as cs
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import string
from google.cloud import language_v1

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "raw-data-392614-6e7d6f0fa9e1.json"

In [2]:
nltk.data.path.append(os.getcwd())
nltk.data.path

['/Users/eric.hoanshelt/nltk_data',
 '/Users/eric.hoanshelt/anaconda3/nltk_data',
 '/Users/eric.hoanshelt/anaconda3/share/nltk_data',
 '/Users/eric.hoanshelt/anaconda3/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data',
 '/Users/eric.hoanshelt/Data Science/My Projects/NLP Systems']

In [5]:
#nltk.download('stopwords') #download stopword corpus
stopwords.words('english')[0:10] # Show some stop words tp verify install

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [6]:
class Website:
    def __init__(self, domain):
        self.domain = domain
        
    def grab_details(self):
        scraper = cs.create_scraper()
        data = scraper.get(self.domain)
        return data
    
    def text_process(self, content):
        """
        Takes in a string of text, then performs the following:
        1. Remove all punctuation
        2. Remove all stopwords
        3. Returns a list of the cleaned text
        """
        # Check characters to see if they are in punctuation
        nopunc = [char for char in content if char not in string.punctuation]

        # Join the characters again to form the string.
        nopunc = ''.join(nopunc)

        # Now just remove any stopwords
        return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
        
    def grab_content(self):
        try:
            data = self.grab_details()
            h1_all:str = ""
            p_all:str = ""
            h2_all:str = ""
            h3_all:str = ""
            allthecontent:str = ""

            if self.grab_details().status_code != 200:
                return None

            soup = BeautifulSoup(data.text, 'html.parser')
            title = soup.find('title').text
            description = soup.find('meta', attrs={'name': 'description'})

            if "content" in str(description):
                description = description.get("content")
            else:
                description = ""


            h1 = soup.find_all('h1')
            for x in range (len(h1)):
                h1_all = h1_all + " " + h1[x].text
                
            h2 = soup.find_all('h2')
            for x in range (len(h2)):
                h2_all = h2_all + " " + h2[x].text

            h3 = soup.find_all('h3')
            for x in range (len(h3)):
                h3_all = h3_all + " " + h3[x].text

            paragraphs = soup.find_all('p')
            for x in range (len(paragraphs)):
                p_all = p_all + " " + paragraphs[x].text

            allthecontent = title + " " + description + " " + h1_all + " " + h2_all + " " + h3_all + " " + p_all + " "
            allthecontent = self.text_process(allthecontent)
            allthecontent = allthecontent[0:999]
            allthecontent = " ".join(allthecontent)
            return allthecontent

        except Exception as e:
            print(e)

In [7]:
def classify(urlList:list):
    result = pd.DataFrame(columns=['URL', "Text", "Category", "Confidence"])
    language_client = language_v1.LanguageServiceClient()
    
    for url in urlList:
        website = Website(url)
        content = website.grab_content()

        document = language_v1.Document(
            content=content, type_=language_v1.Document.Type.PLAIN_TEXT
        )
        response = language_client.classify_text(request={"document": document})
        categories = response.categories

        for category in categories:
            result = result.append({"URL":url, "Text":content, "Category":category.name, "Confidence":category.confidence}, ignore_index=True)
        

    return result

In [8]:
urls = ["https://www.disney.com", "https://wpengine.com", "https://deltiasgaming.com", "https://velocitize.com"]

classification = classify(urls)

/var/folders/67/pb81vw6150zg1p9ptc37j0xr0000gn/T/ipykernel_2696/1517169264.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"URL":url, "Text":content, "Category":category.name, "Confidence":category.confidence}, ignore_index=True)
/var/folders/67/pb81vw6150zg1p9ptc37j0xr0000gn/T/ipykernel_2696/1517169264.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"URL":url, "Text":content, "Category":category.name, "Confidence":category.confidence}, ignore_index=True)
/var/folders/67/pb81vw6150zg1p9ptc37j0xr0000gn/T/ipykernel_2696/1517169264.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"URL":url, "Text":content, "Category":category.name, "Confidenc

In [9]:
classification

,URL,Text,Category,Confidence
0,https://www.disney.com,Disneycom official home things Disney official...,/Travel/Tourist Destinations/Theme Parks,0.81
1,https://www.disney.com,Disneycom official home things Disney official...,/Arts & Entertainment,0.79
2,https://www.disney.com,Disneycom official home things Disney official...,/People & Society/Kids & Teens,0.51
3,https://wpengine.com,Trusted WordPress Platform 2023 WP Engine® WP ...,/Online Communities/Blogging Resources & Services,0.92
4,https://wpengine.com,Trusted WordPress Platform 2023 WP Engine® WP ...,/Internet & Telecom/Web Services,0.81
5,https://deltiasgaming.com,Home Page Deltias Gaming Deltiasgaming ultimat...,/Games/Roleplaying Games,0.97
6,https://deltiasgaming.com,Home Page Deltias Gaming Deltiasgaming ultimat...,/Games/Computer & Video Games,0.59
7,https://velocitize.com,Velocitize fuel digital success fuel digital s...,/Computers & Electronics/Enterprise Technology,0.58
8,https://velocitize.com,Velocitize fuel digital success fuel digital s...,/Internet & Telecom/Web Services,0.58
9,https://velocitize.com,Velocitize fuel digital success fuel digital s...,/Business & Industrial/Business Services/E-Com...,0.50


In [29]:
clean = classification.groupby("URL",as_index=False).max()
clean[clean["Confidence"] > .8]

,URL,Text,Category,Confidence
0,https://deltiasgaming.com,Home Page Deltias Gaming Deltiasgaming ultimat...,/Games/Roleplaying Games,0.97
2,https://wpengine.com,Trusted WordPress Platform 2023 WP Engine® WP ...,/Online Communities/Blogging Resources & Services,0.92
3,https://www.disney.com,Disneycom official home things Disney official...,/Travel/Tourist Destinations/Theme Parks,0.81
